In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

In [ ]:
!pip install selenium
!pip install webdriver_manager
# webdriver_manager는 4버전 이상에서 크롬 버전을 자동으로 관리

In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

# Crawling

## 동적크롤링

reference: https://selenium-python.readthedocs.io/index.html

### 설치 방법

```cmd
pip install selenium
```

Chrome Driver: https://chromedriver.chromium.org/downloads

### 사용법

기본적인 사용 방법은 아래와 같음
```python
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
 
driver = webdriver.Chrome(executable_path='chromedriver') # executable_path: chromedriver path
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))    # 4버전 이상
# driver = webdriver.Firefox()

driver.get(url="http://www.naver.com")

driver.close()
```

#### Locating Elements

find_element: 조건에 해당하는 하나의 값 반환  
find_elements: 조건에 해당하는 다수의 값 반환

element를 찾는 method는 아래와 같음
- by_ID
- by_xpath
- by_link_text
- by_partial_link_text
- by_name
- by_tag_name
- by_class_name
- by_css_selector

```python
element = driver.find_element(By.XPATH, x_path)
element = driver.find_element_by_xpath(x_path)

element = driver.find_element(By.CSS_SELECTOR, css)
element = driver.find_element_by_css_selector(css)
```

해당 값을 못 찾은 경우 NoSuchElementException 발생

##### Send Keys
```python
driver.find_element_by_xpath(xpath).send_keys(id)
driver.find_element_by_xpath(xpath).send_keys(password)
driver.find_element_by_xpath(xpath).click()
```

외에도 Keys 내의 다양한 커맨드 입력 가능
```
from selenium.webdriver.common.keys import Keys
```
|Command          |Action |
|-----------------|-------|
|Keys.ENTER       |엔터    |
|Keys.RRETURN     |       |
|Keys.SPACE       |스페이스 |
|Keys.ARROW_UP    |화살표   |
|Keys.ARROW_DOWN  |       |
|Keys.ARROW_LEFT  |       |
|Keys.ARROW_RIGHT |       |
|Keys.BACK_SPACE  |지우기   |
|Keys.DELETE      |       |
|Keys.CONTROL     |Ctrl   |
|Keys.ALT         |Alt    |
|Keys.SHIFT       |Shift  |
|Keys.TAB         |Tab    |
|Keys.PAGE_UP     |Page-Up|
|Keys.PAGE_DOWN   |Page-Down|
|Keys.TAB         |Tab    |
|Keys.F1 ~ Keys.F9|F1~F9  |
|Keys.ESCAPE      |ESC    |
|Keys.HOME        |Home   |
|Keys.INSERT      |Insert |
|...              |...    |

##### Drag and Drop

```python
from selenium.webdriver import ActionChains

element = driver.find_element(By.NAME, "source")
target = driver.find_element(By.NAME, "target")

action_chains = ActionChains(driver)
action_chains.drag_and_drop(element, target).perform()
```

##### Moving between Windows

```python
previous_window = driver.window_handles[0]
new_window = driver.window_handles[1]
driver.switch_to.window(new_window)
```


##### Moving

```python
driver.forward()
driver.back()
```

#### Waiting

Selenium은 실행 중인 driver를 이용하여 정보를 추출  
driver에 데이터 로딩이 완료되지 않을 경우 데이터 수집이 불가  
따라서 페이지 이동 등의 작업이 들어가면 다음 작업이 완료될 때까지 대기해야만 함

##### Time

가장 기본적인 방법으로 대기 시간을 time 모듈을 통해 명시

```python
import time
from selenium import webdriver


driver = webdriver.Chrome(executable_path='chromedriver')
driver.get(url="http://www.naver.com")
time.time(5)
driver.close()
```

##### Implicit Wait

위의 time에서는 2가지 문제가 존재
1. 모든 반응형 동작마다 time을 걸어 웹 로딩 대기
2. time을 5초로 설정했지만 실제 그보다 동작이 빨리 끝날 경우 불필요한 시간 대기

위의 문제를 해결하고자 implicit wait을 이용  
이는 driver의 옵션 설정으로 반응형 동작에서 최대 허용 대기 시간을 설정  
만약 허용 시간 이내에 로드가 완료될 경우 다음 작업을 바로 진행

```python
from selenium import webdriver


driver = webdriver.Chrome(executable_path='chromedriver')
driver.implicitly_wait(time_to_wait=5)
driver.get(url="http://www.naver.com")
driver.close()

```

##### Explicit Wait

implicit wait에서는 driver가 로딩을 대기하는 시간을 설정하여 5초를 설정하였어도 이전에 로드가 완료되면 다음 작업의 진행이 가능하였음  
하지만 위 경우도 문제가 있는데 5초가 지나도 내가 원하는 데이터가 로드가 안 되었을 경우 데이터를 수집할 수 없음  
따라서 특정 값이 로드될 때까지 기다렸다가 그 값이 로드되면 다음 작업을 진행할 필요성이 발생  
이 때 이용하는 게 explicit wait

```python
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

driver = webdriver.Chrome(executable_path='chromedriver') 
driver.get(url="http://www.naver.com")

try:
    element = WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.CLASS_NAME , 'paging'))
    )
finally:
    driver.quit()

driver.close()

```

until: 조건이 False인 동안에 계속 실행  
not_until: 조건이 True인 동안에 계속 실행

#### ActionChains

연속 동작을 수행하기 위함

ex) control + c
```python
ActionChains(driver).key_down(Keys.CONTROL).send_keys('c').key_up(Keys.CONTROL).perform()
```

##### Drag and Drop

```python
from selenium.webdriver import ActionChains

element = driver.find_element(By.NAME, "source")
target = driver.find_element(By.NAME, "target")

action_chains = ActionChains(driver)
action_chains.drag_and_drop(element, target).perform()
```

#### Others

##### Options
```python
options = webdriver.ChromeOptions()
options.add_argument('window-size=1920,1080')

driver = webdriver.Chrome(executable_path, options=options)
```

##### Alert

경고창 발생 시 이에 수락, 거절 등의 행동을 취할 수 있음

```python
from selenium.webdriver.common.alert import Alert

Alert(driver).accept() # 수락
Alert(driver).dismiss() # 거절
Alert(driver).send_keys(keysToSend=key) # 특정 키를 보낼 수 있음
```

##### Scroll Down
페이지의 최하단으로 이동
```python
driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
```
<br>

특정 태그가 등장할 때까지 이동
```python
from selenium.webdriver import ActionChains

some_tag = driver.find_element_by_id('gorio')
ActionChains(driver).move_to_element(some_tag).perform()
```

##### Minimize/Maximize

```python
driver.minimize_window()
driver.maximize_window()
```

##### Screen Shot
```python
driver.save_screenshot('screenshot.png')
```

# Practice